In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()

2024-04-01 19:06:39.448674: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 19:06:39.448746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 19:06:39.452277: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-01 19:06:39.473028: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-01 19:06:42.084098: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels



# Get list of image paths
image_paths = list(paths.list_images("../LC25000/"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/25000
[INFO]: Processed 2000/25000
[INFO]: Processed 3000/25000
[INFO]: Processed 4000/25000
[INFO]: Processed 5000/25000
[INFO]: Processed 6000/25000
[INFO]: Processed 7000/25000
[INFO]: Processed 8000/25000
[INFO]: Processed 9000/25000
[INFO]: Processed 10000/25000
[INFO]: Processed 11000/25000
[INFO]: Processed 12000/25000
[INFO]: Processed 13000/25000
[INFO]: Processed 14000/25000
[INFO]: Processed 15000/25000
[INFO]: Processed 16000/25000
[INFO]: Processed 17000/25000
[INFO]: Processed 18000/25000
[INFO]: Processed 19000/25000
[INFO]: Processed 20000/25000
[INFO]: Processed 21000/25000
[INFO]: Processed 22000/25000
[INFO]: Processed 23000/25000
[INFO]: Processed 24000/25000
[INFO]: Processed 25000/25000
(25000, 224, 224, 3)
(25000,)


In [8]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))
model = Model(inputs=base_model.input, outputs=base_model.output)
# Extract features
features = model.predict(data, batch_size=32, verbose=1)
 
# Flatten the features
features_flatten = features.reshape(features.shape[0], -1)
    
print(features_flatten.shape)

2024-04-01 19:18:43.500158: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 36183 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0
2024-04-01 19:19:07.175620: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-04-01 19:19:07.501963: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


782/782 [==============================] - 20s 19ms/step
(25000, 25088)


In [9]:
data = features_flatten

In [10]:
num_classes = len(np.unique(labels))

In [11]:
import gc
import os

def calculate_sensitivity_specificity(y_true, y_pred, class_label):
    # Create a binary confusion matrix for the specified class
    true_positive = sum((y_true == class_label) & (y_pred == class_label))
    false_positive = sum((y_true != class_label) & (y_pred == class_label))
    true_negative = sum((y_true != class_label) & (y_pred != class_label))
    false_negative = sum((y_true == class_label) & (y_pred != class_label))

    # Calculate sensitivity and specificity
    sensitivity = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    specificity = true_negative / (true_negative + false_positive) if (true_negative + false_positive) > 0 else 0

    return sensitivity, specificity
# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "EfficientNetB0" or name == "ResNet50":
        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = to_categorical(labels[train_index]), to_categorical(labels[test_index])

                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                #train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                #train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])


                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                # print(y_test)
                # print(y_test_encoded)
                # print(y_pred)
                # print(predictions)
                # Calculate evaluation metrics for the current fold
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivities = []
                specificities = []
                for class_label in range(num_classes):
                    sen, spe = calculate_sensitivity_specificity(y_test_encoded, y_pred, class_label)
                    sensitivities.append(sen)
                    specificities.append(spe)
                sensitivity = sum(sensitivities) / num_classes
                specificity = sum(specificities) / num_classes
                auc_roc = roc_auc_score(y_test, predictions,multi_class='ovr')
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, average='macro')
                f1 = f1_score(y_test_encoded, y_pred, average='macro')
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_prob = model.predict_proba(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivities = []
            specificities = []
            for class_label in range(num_classes):
                sen, spe = calculate_sensitivity_specificity(y_test, y_pred, class_label)
                sensitivities.append(sen)
                specificities.append(spe)
            sensitivity = sum(sensitivities) / num_classes
            specificity = sum(specificities) / num_classes
            auc_roc = roc_auc_score(y_test, y_prob,multi_class='ovr')
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='macro')
            f1 = f1_score(y_test, y_pred, average='macro')
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [12]:
from sklearn.linear_model import LogisticRegression
# SVM
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)




[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9856
Sensitivity: 0.9856000000000001
Specificity: 0.9964000000000001
AUC-ROC: 0.9995042000000002
MCC: 0.9820078560942732
Precision: 0.9856164899055383
F1 Score: 0.9855925400292099

[INFO] Fold 2 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9844
Sensitivity: 0.9843999999999999
Specificity: 0.9961
AUC-ROC: 0.9996179999999999
MCC: 0.9805062752602426
Precision: 0.9844735401774012
F1 Score: 0.9844241659419293

[INFO] Fold 3 / 10 for LR


...

Accuracy: 0.9884
Sensitivity: 0.9884000000000001
Specificity: 0.9971
AUC-ROC: 0.9996074
MCC: 0.98550453333128
Precision: 0.988409613302826
F1 Score: 0.9883957924868156

[INFO] Fold 9 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9812
Sensitivity: 0.9812
Specificity: 0.9953
AUC-ROC: 0.9994562
MCC: 0.976503710721151
Precision: 0.9812493738762644
F1 Score: 0.9812171251898925

[INFO] Fold 10 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.986
Sensitivity: 0.986
Specificity: 0.9964999999999999
AUC-ROC: 0.9995476
MCC: 0.9825053055429749
Precision: 0.9860323429474785
F1 Score: 0.9860054768736317


In [13]:
#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.8392
Sensitivity: 0.8392
Specificity: 0.9597999999999999
AUC-ROC: 0.9037883000000001
MCC: 0.7994324500351047
Precision: 0.842248662061154
F1 Score: 0.8399094338002768

[INFO] Fold 2 / 10 for NB
Accuracy: 0.812
Sensitivity: 0.8119999999999999
Specificity: 0.953
AUC-ROC: 0.8896773
MCC: 0.7654656978992861
Precision: 0.815415029084279
F1 Score: 0.8127878626804087

[INFO] Fold 3 / 10 for NB
Accuracy: 0.8272
Sensitivity: 0.8272
Specificity: 0.9568
AUC-ROC: 0.8957503000000001
MCC: 0.7845470199758678
Precision: 0.8302758806650502
F1 Score: 0.8276475880737738

[INFO] Fold 4 / 10 for NB
Accuracy: 0.8296
Sensitivity: 0.8296000000000001
Specificity: 0.9574
AUC-ROC: 0.9004628
MCC: 0.7872253361414211
Precision: 0.8310166922426637
F1 Score: 0.8298364825651736

[INFO] Fold 5 / 10 for NB
Accuracy: 0.8236
Sensitivity: 0.8236000000000001
Specificity: 0.9559
AUC-ROC: 0.8972892
MCC: 0.7797352036114641
Precision: 0.8254426928037987
F1 Score: 0.8240770293478169

[INFO] 

In [14]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.746
Sensitivity: 0.746
Specificity: 0.9365
AUC-ROC: 0.9530291999999999
MCC: 0.7059949112158864
Precision: 0.8245576823307943
F1 Score: 0.7528987684765445

[INFO] Fold 2 / 10 for KNN
Accuracy: 0.732
Sensitivity: 0.732
Specificity: 0.933
AUC-ROC: 0.9526133
MCC: 0.6910588392686907
Precision: 0.8197122938729796
F1 Score: 0.7390880615425496

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.728
Sensitivity: 0.728
Specificity: 0.932
AUC-ROC: 0.9511322999999999
MCC: 0.6863990397009593
Precision: 0.8155079168101682
F1 Score: 0.7344411007193428

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.756
Sensitivity: 0.756
Specificity: 0.9390000000000001
AUC-ROC: 0.9588609
MCC: 0.7174395993138536
Precision: 0.833757691238123
F1 Score: 0.7626718710296688

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.7404
Sensitivity: 0.7404
Specificity: 0.9350999999999999
AUC-ROC: 0.9530481999999999
MCC: 0.6990942360216972
Precision: 0.8211961418763438
F1 Score: 0.7481108092546289

[INFO] Fold 6 / 10 for K

In [15]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest
Accuracy: 0.9576
Sensitivity: 0.9576
Specificity: 0.9894000000000001
AUC-ROC: 0.9965329
MCC: 0.9470280324446467
Precision: 0.9576413202190457
F1 Score: 0.957562833114672

[INFO] Fold 2 / 10 for Random Forest
Accuracy: 0.9488
Sensitivity: 0.9488000000000001
Specificity: 0.9872
AUC-ROC: 0.9961764000000001
MCC: 0.9360248985934582
Precision: 0.9487565189645644
F1 Score: 0.94872697278291

[INFO] Fold 3 / 10 for Random Forest
Accuracy: 0.9556
Sensitivity: 0.9556000000000001
Specificity: 0.9888999999999999
AUC-ROC: 0.9959843000000002
MCC: 0.9445170014590408
Precision: 0.955503402987412
F1 Score: 0.9555172778001401

[INFO] Fold 4 / 10 for Random Forest
Accuracy: 0.9624
Sensitivity: 0.9623999999999999
Specificity: 0.9906
AUC-ROC: 0.9964399
MCC: 0.953034500473387
Precision: 0.962353146604141
F1 Score: 0.9623070091901905

[INFO] Fold 5 / 10 for Random Forest
Accuracy: 0.952
Sensitivity: 0.952
Specificity: 0.9879999999999999
AUC-ROC: 0.9964055999999999
MCC: 0.

In [16]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 8.719257 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4824862
[LightGBM] [Info] Number of data points in the train set: 22500, number of used features: 24402
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
Accuracy: 0.9776
Sensitivity: 0.9776
Specificity: 0.9944
AUC-ROC: 0.9990622
MCC: 0.9720184685263545
Precision: 0.9776951698598007
F1 Score: 0.9776108726048705

[INFO] Fold 2 / 10 for LGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 8.470096 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4828486
[LightGBM] [Info] N

In [17]:
# Save results to a CSV file
results_df.to_csv('DS6_VGG16.csv', index=False)

In [18]:
results_df

,Model,Average Accuracy,Average Sensitivity,Average Specificity,Average AUC-ROC,Average MCC,Average Precision,Average F1 Score,Memory Used (MB),Time (s)
0,LR,0.98668,0.98668,0.99667,0.999591,0.983358,0.986719,0.986683,-1405.164062,4762.687823
1,NB,0.82396,0.82396,0.95599,0.895728,0.780341,0.826587,0.824507,2153.765625,157.262033
2,KNN,0.74200,0.74200,0.93550,0.953484,0.701370,0.823304,0.749083,2164.097656,578.332490
3,Random Forest,0.95452,0.95452,0.98863,0.996352,0.943179,0.954496,0.954449,2234.886719,279.479823
4,LGBM,0.97688,0.97688,0.99422,0.998974,0.971116,0.976971,0.976895,2797.761719,16000.373827
